На этом занятии мы попробуем задачу регрессии: будем предсказывать значение некоторой переменной, основываясь на значениях и поведении других параметров из датасета. 

Данные возьмем вот эти - https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [ ]:
# традиционная ячейка импортов
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


# Подготовка данных

In [90]:
# загрузим датасет
data = pd.read_csv('/Users/macbook/Documents/datasets/imdb/IMDB-Movie-Data.csv')

data.head(5)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [11]:
# давайте посмотрим на информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Genre               1000 non-null   object 
 3   Description         1000 non-null   object 
 4   Director            1000 non-null   object 
 5   Actors              1000 non-null   object 
 6   Year                1000 non-null   int64  
 7   Runtime (Minutes)   1000 non-null   int64  
 8   Rating              1000 non-null   float64
 9   Votes               1000 non-null   int64  
 10  Revenue (Millions)  872 non-null    float64
 11  Metascore           936 non-null    float64
dtypes: float64(3), int64(4), object(5)
memory usage: 93.9+ KB


In [82]:
# а сейчас -- проверим на NaN
data.isna().any()

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)     True
Metascore              True
dtype: bool

## Что делать с NaN?
Есть 3 варианта

### 1. Убрать строки с NaN

In [83]:
data

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0


In [84]:
print(f"размер оригинального датасета:{data.shape},\
        \nразмер датасета без NaN:{data.dropna().shape}")

# проверяем присутствие NaN
# data_imdb_1.isna().any()

размер оригинального датасета:(1000, 12),        
размер датасета без NaN:(838, 12)


### 2. Превратить NaN в 0

In [86]:
print(f"размер оригинального датасета:{data.shape},\
        \nразмер датасета без NaN:{data.fillna(0).shape}")

# проверяем присутствие NaN
data.fillna(0).isna().any()

размер оригинального датасета:(1000, 12),        
размер датасета без NaN:(1000, 12)


Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
dtype: bool

### 3. Превратить NaN в средние значения по колонке

Таких колонок в нашем датасете две: ```Revenue```  и ``` Metascore```

In [91]:
# вычисляем средние для колонок с пустыми значениями
metascore_mean = data['Metascore'].mean()
revenue_mean = data['Revenue (Millions)'].mean()

#заменяем NaN на средние значения
data['Metascore'].fillna(metascore_mean, inplace=True)

data['Revenue (Millions)'].fillna(revenue_mean, inplace=True)

print(f"размер оригинального датасета:{data.shape}")

# проверяем присутствие NaN
data.isna().any()

размер оригинального датасета:(1000, 12)


Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
dtype: bool

## От NaN избавились, что дальше? Определяем таргет и параметры(фичи)

Попробуем предсказывать год фильма *(это и будет таргет)* по данным его описания, рейтинга, длины в минутах и кассовых сборов *(эти колонки будут фичами)*

In [92]:
data.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [28]:
data.Description

0      A group of intergalactic criminals are forced ...
1      Following clues to the origin of mankind, a te...
2      Three girls are kidnapped by a man with a diag...
3      In a city of humanoid animals, a hustling thea...
4      A secret government agency recruits some of th...
                             ...                        
995    A tight-knit team of rising investigators, alo...
996    Three American college students studying abroa...
997    Romantic sparks occur between two dance studen...
998    A pair of friends embark on a mission to reuni...
999    A stuffy businessman finds himself trapped ins...
Name: Description, Length: 1000, dtype: object

**Подготовим описания фильмов:** для каждого текста в датафрейме:
- приведем в нижний регистр
- уберем пунктуацию
- (наивно)токенизируем
- сохраним значения в отдельную колонку ```text```

PS: почему наивно?

In [95]:
from string import punctuation 

data["text"] = data.Description.apply(lambda x: x.lower().strip(punctuation).split()) 

data["text"][:7]

0    [a, group, of, intergalactic, criminals, are, ...
1    [following, clues, to, the, origin, of, mankin...
2    [three, girls, are, kidnapped, by, a, man, wit...
3    [in, a, city, of, humanoid, animals,, a, hustl...
4    [a, secret, government, agency, recruits, some...
5    [european, mercenaries, searching, for, black,...
6    [a, jazz, pianist, falls, for, an, aspiring, a...
Name: text, dtype: object

Для обучения алгоритма тексты нужно передать в виде числовых значений. У нас есть несколько способов это сделать: 
- векторизовать Tf-Idf или Count векторайзерами
- векторизовать с помощью Word2Vec/Doc2Vec
- векторизовать чем-нибудь еще (BERT, etc) 

В рамках этого эксперимента мы попробуем вариант 2: будем использовать Doc2Vec 

In [93]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

Doc2Vec требует предобработки данных: с помощью объекта TaggedDocument нам нужно будет специальным образом обработать каждый текст. Чтобы удобно это сделать, сделаем такие шаги:

1) создадим лист элементов, где каждый элемент -- это лист слов в тексте описания конкретного фильма. Получится лист листов.

2) с помощью цикла for каждый элемент этого листа (каждый текст, в свою очередь представленный как лист), обработаем с помощью TaggedDocument

In [42]:
# создаем лист листов(описаний)
input_text = list(data.text.values) 

In [105]:
input_text[9]

['a',
 'spacecraft',
 'traveling',
 'to',
 'a',
 'distant',
 'colony',
 'planet',
 'and',
 'transporting',
 'thousands',
 'of',
 'people',
 'has',
 'a',
 'malfunction',
 'in',
 'its',
 'sleep',
 'chambers.',
 'as',
 'a',
 'result,',
 'two',
 'passengers',
 'are',
 'awakened',
 '90',
 'years',
 'early']

In [106]:
documents[9]

TaggedDocument(words=['a', 'spacecraft', 'traveling', 'to', 'a', 'distant', 'colony', 'planet', 'and', 'transporting', 'thousands', 'of', 'people', 'has', 'a', 'malfunction', 'in', 'its', 'sleep', 'chambers.', 'as', 'a', 'result,', 'two', 'passengers', 'are', 'awakened', '90', 'years', 'early'], tags=[9])

In [99]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]

# посмотрим на пример разметки
documents[10]

TaggedDocument(words=['the', 'adventures', 'of', 'writer', 'newt', 'scamander', 'in', 'new', "york's", 'secret', 'community', 'of', 'witches', 'and', 'wizards', 'seventy', 'years', 'before', 'harry', 'potter', 'reads', 'his', 'book', 'in', 'school'], tags=[10])

### Теперь векторизуем тексты
Инициализируем модель  Doc2Vec, обучаем на текстах описаний фильмов 

PS: можно поизменять параметры

In [107]:
model_D2V = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [108]:
model_D2V.save("D2V.model") # а так модель можно сохранить

**Готово: теперь тексты представлены в числовом варианте.** 

Каждый текст-описание фильма представлен в виде 5-компонентного вектора. Давайте посмотрим на один из них.

In [120]:
# индекс [] около documents -- это индекс текста из датасета


# вот векторы для описания первого фильма  из датафрейма
model_D2V.dv[documents[999].tags[0]]


array([-0.05782723,  0.17492925,  0.27828383, -0.3689967 ,  0.08098073],
      dtype=float32)

### Возвращаемся к остальным параметрам датасета: 
Помимо текста у нас есть информация о:
- рейтинге фильма
- длине в минутах
- кассовых сборах


Теперь нужно добавить векторы текстов-описаний фильмов в датасет с остальными параметрами

In [121]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model_D2V.dv[x.tags][0])
    vectors.append(vec)

In [122]:
# так получим датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5"])

split_df

,v1,v2,v3,v4,v5
0,-0.021970,0.081084,0.010817,-0.049518,0.050657
1,0.096219,0.030333,0.129640,-0.449377,0.043599
2,0.188868,0.412429,0.250542,-0.415863,-0.087900
3,-0.020050,0.422464,0.588586,-0.620902,-0.101816
4,0.044364,0.342408,0.203026,-0.273031,0.075460
...,...,...,...,...,...
995,0.297910,0.522668,0.359263,-0.436316,-0.167698
996,0.007023,0.476126,0.427430,-0.511750,0.076711
997,-0.091936,0.360829,0.161259,-0.024922,-0.180528
998,-0.038986,0.178545,0.217625,-0.117914,-0.043959


In [123]:
# теперь добавим его к основному датафрейму
data= data.join(split_df, how='left')
print(data.shape)
data.head()

(1000, 18)


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,text,v1,v2,v3,v4,v5
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,"[a, group, of, intergalactic, criminals, are, ...",-0.021970,0.081084,0.010817,-0.049518,0.050657
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,"[following, clues, to, the, origin, of, mankin...",0.096219,0.030333,0.129640,-0.449377,0.043599
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,"[three, girls, are, kidnapped, by, a, man, wit...",0.188868,0.412429,0.250542,-0.415863,-0.087900
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,"[in, a, city, of, humanoid, animals,, a, hustl...",-0.020050,0.422464,0.588586,-0.620902,-0.101816
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,"[a, secret, government, agency, recruits, some...",0.044364,0.342408,0.203026,-0.273031,0.075460


In [157]:
# можно переопределить датасет, оставив только важное

# features = data[['Runtime (Minutes)',"Year",
#                 'Rating', 'Votes',
#                 'Revenue (Millions)','Metascore',"v1","v2","v3","v4","v5"]
#               ]


features.head(5)

,Runtime (Minutes),Year,Rating,Votes,Revenue (Millions),Metascore,v1,v2,v3,v4,v5
0,121,2014,8.1,757074,333.13,76.0,-0.021970,0.081084,0.010817,-0.049518,0.050657
1,124,2012,7.0,485820,126.46,65.0,0.096219,0.030333,0.129640,-0.449377,0.043599
2,117,2016,7.3,157606,138.12,62.0,0.188868,0.412429,0.250542,-0.415863,-0.087900
3,108,2016,7.2,60545,270.32,59.0,-0.020050,0.422464,0.588586,-0.620902,-0.101816
4,123,2016,6.2,393727,325.02,40.0,0.044364,0.342408,0.203026,-0.273031,0.075460


## Теперь разделим данные на части: для обучения и для теста

Признаки положим в матрицу признаков ```X```,  а таргет -- в вектор ```y```<br>
Затем каждую из них разделим на 2 части

In [162]:
# определяем X и y здесь

X = features.drop(["Year"],axis=1) 

X.shape
X.head()

y = data["Year"]


In [130]:
# Разделим датасет на 4 части 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

# посмотрим на размерности
print(f"X_train shape:{X_train.shape},\
      X_test shape:{X_test.shape}")

X_train shape:(750, 10),      X_test shape:(250, 10)


In [133]:
print(f"y_train shape:{y_train.shape},\
      y_test shape:{y_test.shape}")

y_train shape:(750,),      y_test shape:(250,)


# Обучение алгоритма регрессии: Эксперимент 1

In [137]:
# задаем модель регрессора
# силу регуляризации можно варьировать параметром alpha
regressor_1 = Ridge(alpha=10.0) 

# обучаем
regressor_1.fit(X_train, y_train)

Ridge(alpha=10.0)

In [138]:
# давайте предскажем результат для тестовой выборки

y_preds = regressor_1.predict(X_test)

### оценка результатов алгоритма

В качестве метрики будем использовать [среднюю абсолютную ошибку](https://www.youtube.com/watch?v=ZejnwbcU8nw). Она показывает отклонение от правильного ответа в тех же единах измерения

*(а вообще есть [разные способы](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b))*

In [139]:
mean_absolute_error(y_test, y_preds) 

2.3388855743447095

Попробуйте разные значения для параметра регуляризации alpha при обучении модели. Как они влияют на величину ошибки?

# Эксперимент 2

Иногда бывает полезно [нормализовать](https://en.wikipedia.org/wiki/Normalization_(statistics)) данные: это позволяет исправить ситуацию, когда признаки представлены в разных единацах измерения. 
Для этого используется StandardScaler. 

До нормализации:

In [151]:
X.mean()

Runtime (Minutes)        113.172000
Rating                     6.723200
Votes                 169808.255000
Revenue (Millions)        82.956376
Metascore                 58.985043
v1                         0.141047
v2                         0.359267
v3                         0.382995
v4                        -0.398732
v5                        -0.039457
dtype: float64

In [152]:
# используем стандартизатор
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"обычная матрица параметров:\n {X.mean()}, \n\nпосле нормализации:{X_scaled.mean()}")

обычная матрица параметров:
 Runtime (Minutes)        113.172000
Rating                     6.723200
Votes                 169808.255000
Revenue (Millions)        82.956376
Metascore                 58.985043
v1                         0.141047
v2                         0.359267
v3                         0.382995
v4                        -0.398732
v5                        -0.039457
dtype: float64, 

после нормализации:1.1972645097557688e-16


После:

In [153]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=43)

теперь с данными удобнее работать и обучать

In [158]:
# задаем модель регрессора-2
# силу регуляризации можно варьировать параметром alpha
regressor_2 = Ridge(alpha=50.0) 

# обучаем
regressor_2.fit(X_train, y_train)

Ridge(alpha=50.0)

In [159]:
# давайте предскажем результат для тестовой выборки

y_preds = regressor_2.predict(X_test)

In [160]:
mean_absolute_error(y_test, y_preds)

2.441128835403511

### Эксперимент 3 - Предсказываем не год, а рейтинг

In [163]:
# определяем X и y здесь

X = features.drop(["Rating"],axis=1) 

X.shape
X.head()

y = data["Rating"]


In [164]:
# Разделим датасет на 4 части 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

# посмотрим на размерности
print(f"X_train shape:{X_train.shape},\
      X_test shape:{X_test.shape}")

X_train shape:(750, 10),      X_test shape:(250, 10)


In [165]:
print(f"y_train shape:{y_train.shape},\
      y_test shape:{y_test.shape}")

y_train shape:(750,),      y_test shape:(250,)


# Обучение алгоритма регрессии: Эксперимент 1

In [168]:
# задаем модель регрессора
# силу регуляризации можно варьировать параметром alpha
regressor_3 = Ridge(alpha=50.0) 

# обучаем
regressor_3.fit(X_train, y_train)

Ridge(alpha=50.0)

In [169]:
# давайте предскажем результат для тестовой выборки

y_preds = regressor_3.predict(X_test)

### оценка результатов алгоритма

В качестве метрики будем использовать [среднюю абсолютную ошибку](https://www.youtube.com/watch?v=ZejnwbcU8nw). Она показывает отклонение от правильного ответа в тех же единах измерения

*(а вообще есть [разные способы](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b))*

In [170]:
mean_absolute_error(y_test, y_preds) 

0.4626857495120239